In [ ]:
library(tidyverse)
library(lubridate)
library(dplyr)
library(readr)
library(ggplot2)

players <- read_csv("https://raw.githubusercontent.com/Elvis614412/Dsci-100-group-project/refs/heads/main/players.csv")
sessions <- read_csv("https://raw.githubusercontent.com/Elvis614412/Dsci-100-group-project/refs/heads/main/sessions.csv")

merged_data <- left_join(players,sessions, by = "hashedEmail")

merged_data <- merged_data|>
    select(-hashedEmail,-original_start_time,-original_end_time)
    merged_data$Age[is.na(merged_data$Age)] <- mean(merged_data$Age, na.rm = TRUE)

merged_data


To get a large amount of data, total sessions, total play time, and average session duration are key indicators.  
Let’s call these **engagement level metrics**.

1. Relation between "age" and engagement level metrics  
    > Do older players spend more time per session? (age vs average session duration)

2. Relation between "experience" and engagement level metrics  
    > Do players with more experience tend to play more? (experience vs Avergae Played Hours)

3. Relation between "gender" and engagement level metrics  
    > Which gender has more sessions? (gender vs total sessions)

4. Relation between "subscription" and engagement level metrics  
    > Do subscribers tend to play more than non-subscribers? (subscribe vs total play time)


**1. Do older players spend more time per session? (age vs average session duration)**

In [ ]:
merged_data <- merged_data|>
    mutate(
        start_time = as.POSIXct(start_time, format = "%m/%d/%Y %H:%M"),
        end_time = as.POSIXct(end_time, format = "%m/%d/%Y %H:%M"),
        session_duration = as.numeric(difftime(end_time, start_time, units = "mins"))
    )

name_order <- merged_data |>
  count(name, sort = TRUE) |>
  pull(name)

merged_data$name <- factor(merged_data$name, levels = name_order)

merged_data <- merged_data |>
  arrange(name,start_time)

# merged_data <- merged_data |>
#     filter(name == "Morgan") |>
#     arrange(start_time)

merged_data_clean <- merged_data |>
  filter(session_duration <= 1440 ,!is.na(start_time),!is.na(end_time))

merged_data2 <- merged_data_clean |> 
    group_by(name,Age,experience,subscribe) |>
    summarize(average_playtime = mean(session_duration),.groups = "drop")

merged_data2 <- merged_data2 |>
    select(-name,-experience,-subscribe)

merged_data2 <- merged_data2 |>
    mutate(
        Age = as.integer(Age),
        average_playtime = round(average_playtime,2)
    ) |>
    arrange(Age)

average_by_age <- merged_data2|> 
    group_by(Age) |>
    summarize(average_playtime = mean(average_playtime),.groups = "drop")

average_by_age <- average_by_age |> 
    mutate(
        average_playtime = round(average_playtime)
    )

head(average_by_age,5)



To calculate the average playtime per session, changed factor of start_time and end_time. Before calculate it, I found the outliers represented non-overlapping, extremely long sessions and were unlikely to affect the analysis, so we removed them from the dataset.

The outliers were extremly long sessions which is saying that some player played almost a month. I pick specific player "Morgan" and check that he actually played a month or not and found exterm data was error. Few player are also showing same errors so filter if there is a data that represt more than a day (1440 minutes), then filter that data. 

After that, leave needed data (Age, average_playtime) and change their factors as interger and double (round 2) to read data more easliy. 


In [ ]:
library(ggplot2)

age_plot <- average_by_age |>
    ggplot(aes(x = factor(Age), y = average_playtime)) +
    geom_bar(stat = "identity", fill = "skyblue") + 
    labs(
        x = "Age (9 ~ 50)",
        y = "Average Play Time per Session (minutes)",
        title = "Relationship Between Age and Average Play Time Per Session"
    ) +
    theme_minimal() 

age_plot

In [ ]:
merged_data2 <- merged_data2 |>
  mutate(age_group = case_when(
    Age >= 10 & Age < 20 ~ "10s",
    Age >= 20 & Age < 30 ~ "20s",
    Age >= 30 & Age < 40 ~ "30s",
    Age >= 40 & Age <= 59 ~ "40s~50s",
    TRUE ~ NA_character_
  ))

head(merged_data2,5)

grouped_playtime <- merged_data2 |>
  filter(!is.na(age_group)) |>
  group_by(age_group) |>
  summarize(avg_playtime = round(mean(average_playtime), 2))


In [ ]:
age_plot_by_group <- ggplot(grouped_playtime, aes(x = age_group, y = avg_playtime)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  labs(title = "Average Play Time by Age Gruop ",
       x = "Age Group",
       y = "Average Play Time (minutes)") +
  theme_minimal()

age_plot_by_group


Based on the analysis of average playtime by age, **20-year-old players recorded the longest average playtime per session at 150 minutes**, followed by **32-year-olds (116 minutes)** and **24-year-olds (105 minutes)**. 

In contrast, **50-year-olds had the shorest average playtime per session at 5 minutes**, with **45 -year-olds (7 minutes)** and **26-year-olds (15 minutes)** also showing relatively short play durations. 

When grouped by age ranges, **players in their 30s had the highest average playtime per session**, followed by those in their **20s, teens (10s), and 40s~50s.** 

These results indicate that **players in their 20s and 30s tend to spend the most time per game session.**